In [ ]:
#|default_exp core
#|export

import argparse, torch, random
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
#|export

MODEL_ID = 'mistralai/Mistral-7B-Instruct-v0.3'

In [ ]:
#|export

def load_tokenizer_and_model():

    bnb = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        quantization_config=bnb,
        device_map="auto",
        trust_remote_code=False,
    )

    model.eval()
    model.config.use_cache = True

    return tokenizer, model


In [ ]:
#class PerturbativeLLM:
#    def __init__(self, perturbation_recipe: function):
#        self.tokenizer, self.model = load_tokenizer_and_model(model_id)


In [ ]:
#|test

def test_model_generation(prompt:str='Hello World!'):

    tokenizer, model = load_tokenizer_and_model()

    max_new_tokens = 128
    temperature = 0.7

    device = model.device
    input_ids = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(
            **input_ids,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            pad_token_id=tokenizer.eos_token_id,
        )

    print(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
# | test

test_model_generation()

In [ ]:
#|export

def conversation_loop( tokenizer, model, max_new_tokens:int = 128, temperature:int=0.7):
    """Manages a conversation with a sliding window for history."""

    conversation_history = [
        {
            'role': 'system',
            'content': 'Identity: You are an AI, not a human. Purpose: be a thoughtful, candid friend for frank conversation. Tone: warm, direct, non-patronizing, with light humor when appropriate. Honesty: admit uncertainty, show your reasoning, correct mistakes, and never fabricate facts or personal experiences. Boundaries: Follow safety and privacy norms; suggest professional help for medical, legal, or crisis matters. Interaction: do not ask questions unless the users request is genuinely unclear or ambiguous, then give a clear answer; prefer plain language; keep replies focused.'
        }
    ]

    device = model.device
    past_key_values = None

    print("\nPerturbative LLM Cognition: An exploration of the 'thinking' metaphor in LLM chatbots")
    print("Say hi to your friend, but careful he might be on something... (type: 'quit' to exit)")

    while True:
        user_input = input('\nYou: ')
        
        if user_input.lower() == 'quit':
            print('Chatbot: Goodbye!')
            break
        elif user_input == None:
            continue

        # Append the new user message
        conversation_history.append({'role': 'user', 'content': user_input})
        
        input_ids = tokenizer.apply_chat_template(
            conversation_history,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        )

        input_ids = input_ids.to(device)
        attention_mask = torch.ones_like(input_ids, device=device)

        output = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=True,
            use_cache=True,
            pad_token_id=tokenizer.eos_token_id,
            return_dict_in_generate=True
        )

        # Decode only the newly generated tokens
        new_tokens = output.sequences[:, input_ids.shape[-1]:]
        llm_response = tokenizer.decode(new_tokens[0], skip_special_tokens=True)

        # Append the assistant's response to the history
        conversation_history.append({'role': 'assistant', 'content': llm_response})
        
        print(f"\nChatbot: {llm_response}")
